In [ ]:
!pip install datasets faiss-cpu langchain duckduckgo-search wikipedia yfinance sentence-transformers requests transformers


In [ ]:
!pip install -U langchain-community  tf-keras

In [ ]:
!pip uninstall keras -y

In [ ]:
!pip list | grep -E "tensorflow|keras"

🧠 Step 1: Setup Ollama + vLLM LLM Wrappers

In [13]:
import requests
from typing import List, Optional
from langchain.llms.base import LLM

import sys
import tf_keras as keras
sys.modules["keras"] = keras

# Now it's safe to import transformers
from transformers import TFAutoModel, AutoTokenizer
from typing import Optional, List
from langchain.llms.base import LLM
from pydantic import BaseModel, Field
import requests

class OllamaLLM(LLM):
    model: str = "hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest"
    endpoint: str = "http://172.16.30.146:11434/api/generate"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        payload = {
            "model": self.model,
            "prompt": prompt,
            "stream": False
        }

        try:
            headers = {"Content-Type": "application/json"}
            response = requests.post(self.endpoint, json=payload, headers=headers)
            response.raise_for_status()

            data = response.json()
            if "response" not in data:
                raise ValueError(f"No 'response' in Ollama result: {data}")
            
            return data["response"]
        except Exception as e:
            print(f"❌ Error in OllamaLLM: {e}")
            return ""

    @property
    def _llm_type(self) -> str:
        return "ollama"

# --- Instantiate Two Models ---

model_chat = "hf.co/scb10x/typhoon2.1-gemma3-4b-gguf:latest"
model_reasoning = "hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest"

llm_chat = OllamaLLM(model=model_chat)
llm_reasoning = OllamaLLM(model=model_reasoning)


# --- Optional: Helper Function to Use Reasoning Model ---
def get_reasoning_answer(prompt: str) -> str:
    return llm_reasoning(prompt)

📚 Step 2: Vector Store Setup (unchanged)

In [14]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from datasets import load_dataset, load_from_disk

# Load datasets (subset for demo)
#data1 = load_dataset('Josephgflowers/Finance-Instruct-500k', split='train[:2000]')
#data2 = load_dataset("airesearch/WangchanThaiInstruct", split='train[:2000]')
#data3 = load_dataset('PowerInfer/LONGCOT-Refine-500K', split='train[:2000]')
#data1.save_to_disk('finance_instruct_500k')
#data2.save_to_disk('WangchanThaiInstruct')
#data3.save_to_disk('longcot-refine-500k')



data1 = load_from_disk("finance_instruct_500k")
data2 = load_from_disk("WangchanThaiInstruct")
data3 = load_from_disk("longcot-refine-500k")


def extract_text(dataset, fields):
    rows = []
    for sample in dataset:
        text = ""
        for field in fields:
            if field in sample and sample[field] and str(sample[field]).strip():
                text += f"{field}: {sample[field]}\n"
        if text.strip():
            rows.append(Document(page_content=text.strip()))
    return rows

# Apply correct field names for each dataset
docs1 = extract_text(data1, fields=['system', 'user', 'assistant'])
docs2 = extract_text(data2, fields=['Instruction', 'Input', 'Output'])
docs3 = extract_text(data3, fields=['prompt', 'response'])
# ✅ Extract and combine documents
docs = docs1 + docs2 + docs3
print(f"Loaded {len(docs)} documents")

# ✅ Split the documents into smaller chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)
print(f"Split into {len(split_docs)} chunks")

# ✅ Embed and save
texts = [doc.page_content for doc in split_docs]
if not texts:
    raise ValueError("No texts to embed — check if documents were loaded and split correctly")

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(split_docs, embedding_model)
#vectorstore.save_local("finance_rag_faiss102")
print("Vectorstore saved successfully.")


Loaded 6000 documents
Split into 85723 chunks
Vectorstore saved successfully.


🔁 Step 3: RAG Chain and Fallback Logic

In [15]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from datasets import load_dataset

from langchain.chains import RetrievalQA
from duckduckgo_search import DDGS
import wikipedia
import yfinance as yf

# Instantiate Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Instantiate Ollama LLMs (chat and reasoning)
model_chat = "hf.co/scb10x/typhoon2.1-gemma3-4b-gguf:latest"
model_reasoning = "hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest"

llm_chat = OllamaLLM(model=model_chat)
llm_reasoning = OllamaLLM(model=model_reasoning)

# Load retriever and build QA chain using chat model
retriever = FAISS.load_local(
    "finance_rag_faiss102",
    embedding_model,
    allow_dangerous_deserialization=True
).as_retriever()

qa = RetrievalQA.from_chain_type(llm=llm_chat, retriever=retriever)



# Fallback tools
def duckduckgo_search(query, max_results=3):
    with DDGS() as ddgs:
        return [r["title"] + " - " + r["href"] for r in ddgs.text(query, max_results=max_results)]

def wikipedia_summary(query):
    try:
        return wikipedia.summary(query, sentences=2)
    except:
        return "No Wikipedia summary found."

def get_stock_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        return stock.info.get('longBusinessSummary', 'No stock summary found.')
    except:
        return "Error loading stock."

Step 4: Run the Final QA Function

In [16]:
# Self-confidence rating using reasoning model
def is_confident_llm(llm, query, answer, threshold=7):
    if not answer or len(answer.strip()) < 30:
        return False

    prompt = f"""
Question: {query}

Answer: {answer}

On a scale from 1 to 10, how confident are you in this answer? Only respond with a number.
""".strip()

    try:
        rating_str = llm(prompt).strip()
        rating = int(''.join(filter(str.isdigit, rating_str)))  # safely extract digits
    except Exception as e:
        print(f"⚠️ Warning: Failed to get confidence rating from LLM: {e}")
        return False

    print(f"LLM self-confidence rating: {rating}")
    return rating >= threshold

In [17]:
# Main QA pipeline with fallback
def answer_question(query, use_reasoning=False):
    if use_reasoning:
        response = llm_reasoning(query)
        print("🧠 Reasoning Answer:\n", response)
        return response

    rag_answer = qa.run(query)
    print("💬 RAG Answer (via Ollama chat model):\n", rag_answer)

    if is_confident_llm(llm_reasoning, query, rag_answer):
        print("✅ Confident in RAG answer (via self-evaluation)")
        return rag_answer
    else:
        print("⚠️ Fallback triggered...")
        duck = duckduckgo_search(query)
        wiki = wikipedia_summary(query)
        fallback_str = f"RAG: {rag_answer}\nDuckDuckGo: {duck}\nWikipedia: {wiki}"
        return fallback_str

🧠 2. Load Questions

In [18]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

# Define model names
model_chat = "hf.co/scb10x/typhoon2.1-gemma3-4b-gguf:latest"
model_reasoning = "hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest"

# Create Ollama LLMs
llm_chat = OllamaLLM(model=model_chat)
llm_reasoning = OllamaLLM(model=model_reasoning)

# Load FAISS vector store and retriever
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
retriever = FAISS.load_local(
    "finance_rag_faiss", embedding_model, allow_dangerous_deserialization=True
).as_retriever(search_kwargs={"k": 10})  # get top 10 for reranking

# Create RAG chain using the chat model
qa_chain = RetrievalQA.from_chain_type(llm=llm_reasoning, retriever=retriever)

🌍 Step 3: Deep Search Fallback (DuckDuckGo + Wikipedia)

In [19]:
from duckduckgo_search import DDGS
import wikipedia

def deep_search_summary(query):
    with DDGS() as ddgs:
        results = [r["body"] for r in ddgs.text(query, max_results=3)]
    try:
        wiki = wikipedia.summary(query, sentences=2)
    except:
        wiki = ""
    return "\n".join(results + [wiki])

📊 Step 4: Answer Generator with Confidence Check

In [20]:
from langchain.schema import Document

def rerank_documents(query: str, documents: list[Document], llm) -> list[Document]:
    context = "\n\n".join([f"Document {i+1}: {doc.page_content}" for i, doc in enumerate(documents)])
    prompt = (
        f"You are a helpful assistant that reranks documents by relevance to the question.\n\n"
        f"Question: {query}\n\n"
        f"{context}\n\n"
        f"Rank the top 3 most relevant documents. Respond only with the list of document numbers in order, e.g.: 2, 1, 3"
    )

    response = llm.predict(prompt)
    #print("Reranker response:", response)

    try:
        indices = [int(x.strip()) - 1 for x in response.split(",") if x.strip().isdigit()]
        return [documents[i] for i in indices if 0 <= i < len(documents)]
    except:
        return documents[:3]  # fallback

import re

def extract_final_answer(text: str) -> str:
    """
    Extracts the final answer from <response>...</response>.
    Returns only A, B, C, D, Rise, or Fall, regardless of format (e.g., "**C:", "A:", etc.)
    """
    match = re.search(r"<response>\s*(.*?)\s*</response>", text, re.DOTALL | re.IGNORECASE)
    if match:
        response_text = match.group(1).strip()
        # Match options like **A: or A: or just A
        option_match = re.match(r"[\*\s]*([A-D]|Rise|Fall)\b", response_text, re.IGNORECASE)
        if option_match:
            return option_match.group(1).capitalize()
    return "Unknown"

def generate_answer(query: str) -> str:
    try:
        # Step 1: Retrieve top-k documents
        retrieved_docs = retriever.get_relevant_documents(query)

        # Step 2: Rerank documents
        reranked_docs = rerank_documents(query, retrieved_docs, llm_reasoning)

        # Step 3: Create RAG prompt manually from reranked docs
        context = "\n\n".join([doc.page_content for doc in reranked_docs])
        prompt = f"""Answer the question using the context below. Be concise and accurate.

Context:
{context}

Question: {query}
Answer:"""

        rag_answer = llm_chat.predict(prompt)

    except Exception as e:
        print(f"Error during reranked RAG: {e}")
        rag_answer = ""

    # Step 4: Confidence check using reasoning model
    if is_confident_llm(llm_chat, query, rag_answer):
        print("✅ Confident RAG answer")
        return extract_final_answer(rag_answer)

    # Step 5: Fallback to deep search if RAG not confident
    fallback_prompt = f"""Given the following question and background info, choose the best option (A, B, C, D, Rise, or Fall). Do not explain.

Background: {context}
Question: {query}
Final answer:"""

    reasoning_answer = llm_reasoning(fallback_prompt)
    print("🧠 Reasoning Answer:", reasoning_answer)
    return extract_final_answer(reasoning_answer)

📋 Step 5: Load Excel, Answer All, Save

In [23]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Load data
df = pd.read_csv("test_wk.csv")  # adjust path
queries = df["query"].tolist()
# Valid answer types
valid_answers = ["A", "B", "C", "D", "RISE", "FALL"]

# Function to clean and extract valid answer
def process_query(q):
    try:
        answer = generate_answer(q)
        answer_clean = answer.strip().upper()

        # Extract first valid keyword
        for va in valid_answers:
            if va in answer_clean:
                return va
        return ""
    except Exception as e:
        print(f"Error processing query: {e}")
        return ""

# Run in parallel
results = []
with ThreadPoolExecutor(max_workers=8) as executor:  # adjust number of workers if needed
    future_to_query = {executor.submit(process_query, q): q for q in queries}
    for future in tqdm(as_completed(future_to_query), total=len(queries)):
        results.append(future.result())

# Save or display
df["predicted_answer"] = results
#print(df[["query", "predicted_answer"]])


  0%|          | 1/499 [00:06<56:57,  6.86s/it]

LLM self-confidence rating: 10
✅ Confident RAG answer


  0%|          | 2/499 [00:30<2:16:32, 16.48s/it]

🧠 Reasoning Answer: <thoughts>

<plan>
<step order="1">Identifying key concepts in the background information</step>
<step order="2">Understanding the context and focus of the question regarding partnerships for economic growth</step>
<step order="3">Determining which option aligns with the focus on leveraging experience and connections</step>
<step order="4">Finalizing the choice based on the identified key concepts and options provided</step>
</plan>

<step>
<title>Identifying key concepts in the background information</title>
<scratch_pad>
The context revolves around maximizing partnerships for economic growth, emphasizing leveraging corporate executive experience and connections.
Key factors include:
- Economic growth
- Partnerships
- Corporate experience

Options to consider:
- A: Barack Obama (historical figure)
- B: James Dyson (entrepreneur)
- C: Damien Hirst (artist)
- D: Mo Farah (athlete)

Focusing on the entrepreneur aspect is key.
</scratch_pad>
<summary>I've identified th

  1%|          | 3/499 [00:39<1:50:07, 13.32s/it]

...

🧠 Reasoning Answer: <thoughts>

<plan>
<step order="1">ทำความเข้าใจโจทย์และตัวเลือก</step>
<step order="2">ประเมินทุกตัวเลือกตามบริบท</step>
<step order="3">หาคำตอบที่เหมาะสมที่สุด</step>
</plan>

<step>
<title>ทำความเข้าใจโจทย์และตัวเลือก</title>
<scratch_pad>
คำถามพูดถึงวิธีการนำเสนอผลิตภัณฑ์เพื่อตอบสนองความต้องการของลูกค้า โดยมีตัวเลือกต่างๆ ที่เกี่ยวข้องกับแนวคิดการส่งเสริมการขาย การจัดจำหน่าย และการโฆษณา
</scratch_pad>
<summary>กำลังวิเคราะห์คำถามและตัวเลือกเพื่อหาคำตอบที่เหมาะสมที่สุดสำหรับการนำเสนอผลิตภัณฑ์ให้ตรงตามความต้องการของลูกค้า</summary>
<next_step>ประเมินทุกตัวเลือกตามบริบท</next_step>
</step>

<step>
<title>ประเมินทุกตัวเลือกตามบริบท</title>
<scratch_pad>
- A: แนวคิดผลิตภัณฑ์ใหม่และการปรับปรุง - ไม่ใช่การนำเสนอ
- B: การขาย - เป็นส่วนหนึ่งของการนำเสนอ แต่ไม่ได้รวมถึงขั้นตอนก่อนหน้านี้
- C: กิจกรรมการโฆษณาและการส่งเสริมการขาย - ถูกต้อง เพราะเกี่ยวข้องกับการสร้างความสนใจในผลิตภัณฑ์และตอบสนองความต้องการ
- D: สถานที่หรือกิจกรรมการจัดจำหน่าย - เกี่ยวข้องกับช่องทางการนำเสนอ แ

100%|█████████▉| 498/499 [57:45<00:04,  4.58s/it]

🧠 Reasoning Answer: <thoughts>

<plan>
<step order="1">Understanding the question</step>
<step order="2">Evaluating the answer choices</step>
<step order="3">Selecting the correct option based on context</step>
</plan>

<step>
<title>Understanding the question</title>
<scratch_pad>
The question asks for an expression that represents price times quantity. This suggests a need to relate the concept of price and quantity in economic terms.
</scratch_pad>
<summary>I'm clarifying what the question is asking, which revolves around the relationship between price and quantity.</summary>
<next_step>Evaluating the answer choices</next_step>
</step>

<step>
<title>Evaluating the answer choices</title>
<scratch_pad>
Options to consider:
- A: International trade gap (not directly related to price times quantity)
- B: Budget deficit (also not directly related to the relationship between price and quantity)
- C: Total revenue (which can be expressed as Price × Quantity)
- D: Price elasticity of deman

100%|██████████| 499/499 [57:47<00:00,  6.95s/it]

🧠 Reasoning Answer: <thoughts>

<plan>
<step order="1">วิเคราะห์บริบทของคำถาม</step>
<step order="2">ประเมินแต่ละตัวเลือกตามหลักการสำคัญ</step>
<step order="3">ระบุตัวเลือกที่ถูกต้องตามทฤษฎีที่เกี่ยวข้องกับพฤติกรรมมนุษย์</step>
<step order="4">สรุปคำตอบสุดท้าย</step>
</plan>

<step>
<title>วิเคราะห์บริบทของคำถาม</title>
<scratch_pad>
เรากำลังสอบถามทฤษฎีทางจิตวิทยาหรือพฤติกรรมศาสตร์ที่เกิดจากการเลือกปฏิบัติตามค่านิยมและความเชื่อ ซึ่งบ่งชี้ถึงการมีอิทธิพลจากสภาพแวดล้อมและบริบท
</scratch_pad>
<summary>ผมกำลังทำความเข้าใจว่าคำถามถามเกี่ยวกับทฤษฎีที่อธิบายพฤติกรรมมนุษย์โดยพิจารณาจากค่านิยมและความเชื่อ ซึ่งเป็นหลักการพื้นฐานในทฤษฎีพฤติกรรมหลายทฤษฎี</summary>
<next_step>ประเมินแต่ละตัวเลือกตามหลักการสำคัญ</next_step>
</step>

<step>
<title>ประเมินแต่ละตัวเลือกตามหลักการสำคัญ</title>
<scratch_pad>
- ตัวเลือก A: Expectancy Theory - เน้นการเชื่อมโยงระหว่างแรงจูงใจและการตอบสนอง แต่ไม่ได้เน้นที่การเลือกปฏิบัติตามค่านิยมและความเชื่อโดยตรง
- ตัวเลือก B: Instrumental Conditioning - เกี่ยวข้องกับการให

In [ ]:
df

In [24]:
#df["answer"] = answers
df.to_csv("answered_questions104.csv", index=False)

In [34]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
ollama_llm = OllamaLLM(model=model_reasoning)
print(ollama_llm("What is the capital of Thailand?"))
print(model_reasoning)

❌ Error in OllamaLLM: 404 Client Error: Not Found for url: http://172.16.30.146:11434/api/generate

hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest


In [13]:
!curl http://172.16.30.146:11434/api/tags

{"models":[{"name":"hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest","model":"hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest","modified_at":"2025-06-25T12:47:49.787578009Z","size":2019378094,"digest":"56b1b463a6bcce24ee938bd2b3abdb81686602f6fb6de3a54030ced8dac56a44","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"3.21B","quantization_level":"unknown"}},{"name":"hf.co/scb10x/typhoon2.1-gemma3-4b-gguf:latest","model":"hf.co/scb10x/typhoon2.1-gemma3-4b-gguf:latest","modified_at":"2025-06-25T09:04:49.1355109Z","size":2489765421,"digest":"37354827f7b37040c83230a9b0252a803408f3a4d7511561a5457c1eb70c045e","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"3.88B","quantization_level":"unknown"}},{"name":"llama3.2:3b","model":"llama3.2:3b","modified_at":"2025-06-25T09:00:32.26182376Z","size":2019393189,"digest":"a80c4f17acd55265feec403c7aef86be0c25983ab27

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
!curl -X POST http://172.16.30.146:11434/api/generate -d '{ "model": "hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest",  "prompt": "What is the capital of Thailand?",  "stream": false}'

{"model":"hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest","created_at":"2025-06-25T22:31:57.812055091Z","response":"\u003cthoughts\u003e\n\n\u003cplan\u003e\n\u003cstep order=\"1\"\u003eIdentify the user's request\u003c/step\u003e\n\u003cstep order=\"2\"\u003eConfirm the capital city of Thailand\u003c/step\u003e\n\u003cstep order=\"3\"\u003eProvide additional context if necessary\u003c/step\u003e\n\u003c/plan\u003e\n\n\u003cstep\u003e\n\u003ctitle\u003eIdentify the user's request\u003c/title\u003e\n\u003cscratch_pad\u003e\nThe user is asking specifically for the capital city of Thailand.\n\u003c/scratch_pad\u003e\n\u003csummary\u003eThe user wants to know the capital of Thailand.\u003c/summary\u003e\n\u003cnext_step\u003eConfirm the capital city of Thailand\u003c/next_step\u003e\n\u003c/step\u003e\n\n\u003cstep\u003e\n\u003ctitle\u003eConfirm the capital city of Thailand\u003c/title\u003e\n\u003cscratch_pad\u003e\nThe capital of Thailand is indeed Bangkok. \n- Locate

In [36]:
payload = {
    "model": "hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest",
    "prompt": "What is the capital of Thailand?",
    "stream": False
}
response = requests.post('http://172.16.30.146:11434/api/generate', json=payload)

In [22]:
results

['', 'B', 'A', 'A', 'B', '', '', 'C', '', 'RISE']